In [1]:
import torch

from torch_geometric.utils import to_dense_adj
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigDataset

In [3]:
# dataset = CVFConfigDataset(
#     "graph_4_config_rank_dataset.csv", "graph_4_edge_index.json", 10
# )
dataset = CVFConfigDataset(
    "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
)
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [4]:
class VanillaGNNLayer(torch.nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_out, bias=False)

    def forward(self, x, adjacency):
        x = self.linear(x)
        # x = torch.sparse.mm(adjacency, x)
        x = torch.matmul(adjacency, x)
        return x

In [5]:
adjacency = to_dense_adj(dataset.edge_index.t().contiguous())[0]
adjacency += torch.eye(len(adjacency))
adjacency = adjacency.unsqueeze(0)
adjacency

tensor([[[1., 1., 1., 1.],
         [1., 1., 0., 0.],
         [1., 0., 1., 0.],
         [1., 0., 0., 1.]]])

In [8]:
class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gnn1 = VanillaGNNLayer(dim_in, dim_h)
        self.gnn2 = VanillaGNNLayer(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, adjacency):
        print("x", x.shape)
        h = self.gnn1(x, adjacency)
        h = torch.relu(h)
        print(h.shape)
        h = self.gnn2(h, adjacency)
        h = torch.relu(h)
        print(h.shape)
        h = self.out(h)
        print(h.shape)
        h = torch.sigmoid(h)
        # h = h.squeeze(-1)
        return h

    def fit(self, data_loader, epochs):
        # criterion = torch.nn.CrossEntropyLoss()
        criterion = torch.nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        for epoch in range(1, epochs + 1):
            avg_loss = 0
            count = 0
            for batch in data_loader:
                x = batch[0]
                # print(x)
                y = batch[1]
                y = (y <= 0).long()
                optimizer.zero_grad()
                out = self(x, adjacency)
                print("output", out, "y", y)
                loss = criterion(out, y)
                avg_loss += loss
                count += 1
                loss.backward()
                optimizer.step()

            print("Loss:", avg_loss / count)


In [9]:
gnn = VanillaGNN(dataset.num_classes, 10, 1)
print(gnn)

gnn.fit(train_loader, epochs=20)

VanillaGNN(
  (gnn1): VanillaGNNLayer(
    (linear): Linear(in_features=4, out_features=10, bias=False)
  )
  (gnn2): VanillaGNNLayer(
    (linear): Linear(in_features=10, out_features=10, bias=False)
  )
  (out): Linear(in_features=10, out_features=1, bias=True)
)
x torch.Size([2, 4, 4])
torch.Size([2, 4, 10])
torch.Size([2, 4, 10])
torch.Size([2, 4, 1])
output tensor([[[0.4196],
         [0.4611],
         [0.4611],
         [0.4494]],

        [[0.3824],
         [0.4370],
         [0.4284],
         [0.4284]]], grad_fn=<SigmoidBackward0>) y tensor([1, 1])


ValueError: Target size (torch.Size([2])) must be the same as input size (torch.Size([2, 4, 1]))